<a href="https://colab.research.google.com/github/gemma24/ISYS5002-assessment/blob/main/Sandbox_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Client's requirements
This notebook able to:
* Prompts for a company's stock market ticker symbol
* Scrapes the Yahoo finance website to extract info on the key company
* Calculate the each executive's expected income tax based on the ATO tax table
* Data can be saved as a CSV or SQLite database
* Follow PEP8 conventions



# Problem Solving Methodology
In the process of problem-solving, there are five key steps to follow:
1. State the problem clearly
2. Describe the input and output information
3. Work the problem by hand 
4. Develop an algorithm
5. Test solution with a variety of data.
# Step 1: State the problem 
* Find out what is ticker symbol
* Find the Yahoo finance wbesite and how to scrape?
* How to access each executive's information?
* How to calculate the each executive's expected income tax?
* How to save a data as a CSV and SQlite database








# Step 2: Describe the input and output information
## Website 

[Yahoo Finance](https://au.finance.yahoo.com/)

## Inputs

* Prompt for a ticker symbol

## Processing

* Get income from webpage
* Calculate Tax (using income)

## Outputs

* Save to CSV file 
* Save to SQLIte 




### Exploring the Website

Stock Ticker Symbol:  A stock symbol is a unique series of letters assigned to a security for trading purposes.  From https://www.investopedia.com/terms/s/stocksymbol.asp

Ticker can be for any company on the exchange

Note: once found page, need to click on the 'Profile' tab.

For TGT the profile page is:

https://au.finance.yahoo.com/quote/TGT/profile?p=TGT

For WOW.AX:

https://au.finance.yahoo.com/quote/WOW.AX/profile?p=WOW.AX


For COL.AX (Coles Group)

https://au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX


# Step 3: Work the problem by hand
# Try to work an example by hand

Here is the table from the website for COL.AX 
https://au.finance.yahoo.com/quote/COL.AX/profile?p=COL.AX&.tsrc=fin-srch

```
Name	Title	Pay	Exercised	Year born
Mr. Steven Cain	MD, CEO & Director	3.33M	N/A	1965
Ms. Leah Weckert	Chief Financial Officer	1.8M	N/A	N/A
Mr. Matthew Swindells	Chief Operations Officer	1.58M	N/A	N/A
Mr. Greg Davis	Chief Exec. - Commercial & Express	1.61M	N/A	N/A
Mr. John Cox	Chief Technology Officer	N/A	N/A	N/A
Mark Howell	Gen. Mang. of Investor Relations	N/A	N/A	N/A
Mr. David Brewster	Chief Legal Officer	N/A	N/A	N/A
Ms. Lisa Ronson	Chief Marketing Officer	N/A	N/A	N/A
Ms. Kris Webb	Chief People Officer	N/A	N/A	N/A
Mr. Thinus Keeve	Chief Sustainability, Property & Export Officer	N/A	N/A	N/A

```

Client clarified we only need Name, Title, Pay

Let consider one row, and only name title and pay

```
Name	              Title	          Pay
Mr. Steven Cain	MD, CEO & Director	3.33M
```

Note, 1st collum is the name, the second is title and thrid is pay.
I need the dollar amount to calculate the tax.
So Pay (or Salray) is 3.33M. Need to extract the 3.33 and multiply is by 1000,000 to give me the dollar amount.  Then use this dollar amount to calculate the tax. If the last character is K, we need to multiply by 1,000.

**TODO:  Work out how get each colum/cell in the table**

### Want to calculate the tax
* Need to get the pay

*Note: We did a calculate tax in a earlier lab*

See: https://github.com/gemma24/my_first_repo/blob/main/tax_calculation.ipynb



```
def calculate_tax(income):
  ''' 
  This function will calculate the tax payable based on the ATO
  '''
  if income<=18200:
    tax=0
  elif 18200<income<=45000:
    tax= 0.19 * (income- 18200)
  elif 45001< income<= 120000:
    tax=5092+0.32*(income-45000)
  elif 120001< income < 180000:
    tax= 29467+0.37*(income-120000)
  elif 180001<= income:
    tax= 51667+0.45*(income-180000)
  return tax
```
income_tax=calculate_tax(pay)





In [1]:
3.33 * 1000000

3330000.0

# Step 4: Develop an Algorithm
1. Get the ticker symbol from the user
2. Use the symbol to goto the propile page
3. Get the Table
* for each row get the pay
* Use pay to calculate tax
* Add tax to to table

4. Save as Csv
5. Save as SQL

In [14]:
# Usually run of Google Coolab, so install unique packages
!pip install kora -q

#load packages
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd

     |████████████████████████████████| 57 kB 2.5 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 


In [13]:
#lets create a function that obtain the ticker symbol from user and then return the profile page.
def build_url():
  ''' This function will build the URL of the profile page'''
  ticker = input("Please type in a ticker symbol ")
  url_pattern = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  result = url_pattern.format(ticker, ticker)
  return result

In [12]:
# Lets try if this function is working
url = build_url()
print(url)

Please type in a ticker symbol col.ax
https://au.finance.yahoo.com/quote/col.ax/profile?p=col.ax


In [15]:
# Get URL
url = build_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source,"html.parser")

Please type in a ticker symbol col.ax


In [16]:
def get_record(row):
  cells = row.find_all('td')
  name = cells[0].text
  title =  cells[1].text
  pay = cells[2].text
  return {'name': name, 'title': title, 'pay': pay}

In [17]:
def get_rows(soup):
  # Get the table!
  table = soup.find('table')

  # get to the row
  body = table.find('tbody')
  rows = body.find_all('tr')
  return rows

In [18]:
def calculate_tax(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
  elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
  elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
  elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)


  return tax_payable

In [19]:
def get_salary(pay):  
  last_char = pay[-1]
  if last_char == 'M':
    pay_as_num = float(pay[0:-1])
    sal = pay_as_num * 1000000
  elif last_char == 'k':
    pay_as_num = float(pay[0:-1])
    sal = pay_as_num * 1000
  else: # Must be a N/A
    sal = 0
  return sal

In [20]:
url = build_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source,'html.parser')
rows = get_rows(soup)

# getting all the records
records = []
for row in rows:
  rec = get_record(row)
  records.append(rec)

# adding the tax colum
for rec in records:
  sal = get_salary(rec['pay'])
  rec['tax'] = calculate_tax(sal)

exec_df = pd.DataFrame.from_records(records)
exec_df

Please type in a ticker symbol col.ax


,name,title,pay,tax
0,Mr. Steven Cain,"MD, CEO & Director",3.33M,1469167.0
1,Ms. Leah Weckert,Chief Financial Officer,1.8M,780667.0
2,Mr. Matthew Swindells,Chief Operations Officer,1.58M,681667.0
3,Mr. Greg Davis,Chief Exec. - Commercial & Express,1.61M,695167.0
4,Mr. John Cox,Chief Technology Officer,N/A,0.0
5,Mark Howell,Gen. Mang. of Investor Relations,N/A,0.0
6,Mr. David Brewster,Chief Legal Officer,N/A,0.0
7,Ms. Lisa Ronson,Chief Marketing Officer,N/A,0.0
8,Ms. Kris Webb,Chief People Officer,N/A,0.0
9,Mr. Thinus Keeve,"Chief Sustainability, Property & Export Officer",N/A,0.0
